<a href="https://colab.research.google.com/github/pragyasrivastava0805/congenial-journey/blob/main/CNN_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [7]:
#Device Configuration
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [8]:
#HyperParameters
num_epochs=4
batch_size=4
learning_rate=0.001

In [9]:
transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
train_dataset=torchvision.datasets.CIFAR10(root='./data',train=True,download=True,transform=transform)
test_dataset=torchvision.datasets.CIFAR10(root='./data',train=False,download=True,transform=transform)
train_loader=torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
test_loader=torch.utils.data.DataLoader(test_dataset,batch_size=batch_size,shuffle=True)
classes=train_dataset.classes
classes

Files already downloaded and verified
Files already downloaded and verified


['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [18]:
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet,self).__init__()
    self.conv1=nn.Conv2d(3,6,5)
    self.pool=nn.MaxPool2d(2,2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16 * 5 * 5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)
  def forward(self,x):
    x=self.pool(F.relu(self.conv1(x)))
    x=self.pool(F.relu(self.conv2(x)))
    x=x.view(-1,16*5*5)
    x=F.relu(self.fc1(x))
    x=F.relu(self.fc2(x))
    x=self.fc3(x)
    return x
model=ConvNet().to(device)
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)

total_steps=len(train_loader)
for epoch in range(num_epochs):
  for i,(images,labels) in enumerate(train_loader):
    images=images.to(device)
    labels=labels.to(device)
    outputs=model(images)
    loss=criterion(outputs,labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if(i+1)%2000==0:
      print(f'Epoch [{epoch+1}/{num_epochs}],Step[{i+1}/{total_steps}],Loss:{loss.item():.4f}')
  print('Finished Training')
  torch.save(model.state_dict(),'model.ckpt')
  with torch.no_grad():
    n_correct=0
    n_samples=0
    n_class_correct=[0 for i in range(10)]
    n_class_samples=[0 for i in range(10)]
    for (images,labels) in test_loader:
      images=images.to(device)
      labels=labels.to(device)
      outputs=model(images)
      _,predicted=torch.max(outputs,1)
      n_samples+=labels.size(0)
      n_correct+=(labels==predicted).sum().item()
      for i in range(batch_size):
        label=labels[i]
        predict=predicted[i]
        if(label==predict):
          n_class_correct[i]+=1
        n_class_samples[i]+=1
    acc=100.0*n_correct/n_samples
    print(len(n_class_samples))
  


Epoch [1/4],Step[2000/12500],Loss:2.3215
Epoch [1/4],Step[4000/12500],Loss:2.2852
Epoch [1/4],Step[6000/12500],Loss:2.2909
Epoch [1/4],Step[8000/12500],Loss:2.3182
Epoch [1/4],Step[10000/12500],Loss:2.3065
Epoch [1/4],Step[12000/12500],Loss:2.1319
Finished Training
10
Epoch [2/4],Step[2000/12500],Loss:3.2575
Epoch [2/4],Step[4000/12500],Loss:1.9576
Epoch [2/4],Step[6000/12500],Loss:1.6043
Epoch [2/4],Step[8000/12500],Loss:1.5116
Epoch [2/4],Step[10000/12500],Loss:1.7453
Epoch [2/4],Step[12000/12500],Loss:2.1947
Finished Training
10
Epoch [3/4],Step[2000/12500],Loss:1.6999
Epoch [3/4],Step[4000/12500],Loss:1.8950
Epoch [3/4],Step[6000/12500],Loss:1.9488
Epoch [3/4],Step[8000/12500],Loss:1.2559
Epoch [3/4],Step[10000/12500],Loss:1.2678
Epoch [3/4],Step[12000/12500],Loss:1.8027
Finished Training
10
Epoch [4/4],Step[2000/12500],Loss:1.4282
Epoch [4/4],Step[4000/12500],Loss:1.1442
Epoch [4/4],Step[6000/12500],Loss:1.1718
Epoch [4/4],Step[8000/12500],Loss:1.2452
Epoch [4/4],Step[10000/12500]